<a href="https://colab.research.google.com/github/wahyunh10/Car-Auction-Prices-Project/blob/main/Car_Auction_Prices_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Car Auction Prices Project**

**In this project, there are 4 steps that I do:**

1. Data Preparation (Data Cleaning)
  * Extract Raw Data
  * Fill and Drop Null Values
  * Standardization Values
  * Load Clean Data to CSV and Local MySQL Database
2. Exploration Data Analysis
  * Find Insight using Matplotlib and Seaborn
  * Outlier Analysis
  * Find Best and Worst Seller using Quantile Analysis
3. Machine Learning Modelling
  * Prediction Prices Cars using Regression
  * Clustering Quality of Cars using KMeans Clustering
4. Hypothesis Testing using MannWhitneyU

The dataset contains historical car auction sales prices, scraped from the outside internet sources collected in 2015. This dataset taken from kaggle (https://www.kaggle.com/tunguz/used-car-auction-prices).

This dataset contains 491641 rows and 15 columns:

* year : information of year production of the car
* make : brand of the car
* model : model of the brand car
* trim : spesific model of the car
* body : body type of the car
* transmission : transmission type of the car
* vin : unique id of the car
* state : code state where the transaction happened
* condition : rating condition of the car
* odometer : total distance of the car
* color : body color of the car
* interior : interior color of the car
* seller : place that sold the car
* mmr : market price of the car
* sellingprice : price of the car
* saledate : date when the car sold